In [1]:
import pandas as pd
import numpy as np

In [2]:
data_set = pd.read_csv('data/people_wiki.csv')

In [3]:
print("n_samples: %d, n_features: %d" % data_set.shape)

n_samples: 59071, n_features: 3


In [4]:
print(data_set.head(10))

                                                 URI                 name  \
0        <http://dbpedia.org/resource/Digby_Morrell>        Digby Morrell   
1       <http://dbpedia.org/resource/Alfred_J._Lewy>       Alfred J. Lewy   
2        <http://dbpedia.org/resource/Harpdog_Brown>        Harpdog Brown   
3  <http://dbpedia.org/resource/Franz_Rottensteiner>  Franz Rottensteiner   
4               <http://dbpedia.org/resource/G-Enka>               G-Enka   
5        <http://dbpedia.org/resource/Sam_Henderson>        Sam Henderson   
6        <http://dbpedia.org/resource/Aaron_LaCrate>        Aaron LaCrate   
7      <http://dbpedia.org/resource/Trevor_Ferguson>      Trevor Ferguson   
8         <http://dbpedia.org/resource/Grant_Nelson>         Grant Nelson   
9         <http://dbpedia.org/resource/Cathy_Caruth>         Cathy Caruth   

                                                text  
0  digby morrell born 10 october 1979 is a former...  
1  alfred j lewy aka sandy lewy graduated 

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
count_vectorizer = CountVectorizer()
count = count_vectorizer.fit_transform(data_set['text'])

In [7]:
analyzer = count_vectorizer.build_analyzer()

In [8]:
voc = count_vectorizer.vocabulary_ 

In [9]:
def count_words(doc):
    dic = {}
    if analyzer(doc):
        M = count_vectorizer.fit_transform([doc]).toarray()[0]
        for word,index in count_vectorizer.vocabulary_.items():
            dic[word] = M[index]
    return dic

In [10]:
data_set['word_count'] = data_set['text'].apply(count_words)

In [11]:
obama = data_set[data_set['name'] == 'Barack Obama']

In [70]:
obama_word_count_table = pd.DataFrame()
obama_word_count_table['words'] = obama['word_count'].values[0].keys()
obama_word_count_table['count'] = obama['word_count'].values[0].values()

In [72]:
obama_word_count_table.sort_values(by='count',ascending=False).head()

,words,count
11,the,40
25,in,30
13,and,21
16,of,18
22,to,14


In [76]:
print(voc['the'])

488148


In [78]:
rvoc = dict(zip(voc.values(),voc.keys()))

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
knn_model = KNeighborsClassifier(algorithm='brute',metric='cosine')
knn_model.fit(count,data_set['name'])

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [89]:
dist,ind= knn_model.kneighbors(count[obama.index[0]])

In [92]:
pd.DataFrame({'reference_label':data_set.iloc[ind[0]]['name'],'distance':dist[0]})

,distance,reference_label
35817,1.443290e-15,Barack Obama
24478,1.224048e-01,Joe Biden
25798,1.357289e-01,Sandro Petrone
28447,1.394896e-01,George W. Bush
20389,1.421223e-01,Ribal al-Assad
